# 误差分析


*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要

本教程将以 Cross Resonance（CR）门为例，演示如何分析实际量子门操作的误差。我们将使用动力学演化分析、真值表（Truth Table）等工具可视化非计算空间的能级泄露等相关信息。本教程的大纲如下：

- 背景介绍
- 准备工作
- 构造哈密顿量并优化脉冲
- 动力学分析
- 真值表
- 总结

## 背景介绍

使用控制脉冲实现目标量子门时，实际的量子操作可能与目标操作不同。两者之间差异通常被量化为失真度。分析造成误差的来源，对我们提高量子门的保真度至关重要。

量脉提供了通过动力学分析来探究量子比特状态总体演化的功能，并通过真值表可视化计算空间外的能级泄露。在本教程中，我们将以 Cross Resonance（CR）量子门为例进行演示。有关 CR 门的更多详细信息，请参见 [Cross Resonance 量子门](https://quanlse.baidu.com/#/doc/tutorial-cr) 教程。

## 准备工作

在设备上成功安装量脉后，您可以按照本教程运行下面的量脉的程序。要运行此教程，您需要从量脉（Quanlse）和其它常用的 Python 库导入以下包：

In [ ]:
# This module is imported for creating Hamiltonian dictionary
from Quanlse.QHamiltonian import QHamiltonian as QHam

# These functions are imported to define useful operators matrices to free us from defining them manually
from Quanlse.QOperator import number, duff

# These functions are imported to helps us perform matrix calculation
from Quanlse.Utils.Functions import basis, tensor, projector

# This function is imported to perform CR gate optimization using Quanlse Cloud Service
from Quanlse.remoteOptimizer import remoteOptimizeCr

# Import the necessary packages
import numpy
import matplotlib.pyplot as plt

在使用 `remoteOptimizeCr()` 函数访问云服务之前，我们需要从[百度量易伏](https://quantum-hub.baidu.com)官网上申请一个 token。

In [ ]:
# Import Define class and set the token
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

## 构造哈密顿量并优化脉冲
我们首先使用以下代码构造系统哈密顿量，并优化控制参数以实现高保真度的 CR 门：

In [ ]:
# Define parameters to initialize the Hamiltonian
dt = 2.0  # Sampling period
qubits = 2  # Number of qubits
level = 3  # Energy level

# Define qubit parameters
g = 0.0038 * (2 * numpy.pi)  # Coupling strength, GHz
wq0  = 4.914 * (2 * numpy.pi)  # Transition frequency for qubit 0, GHz
wq1 = 4.714 * (2 * numpy.pi)  # Transition frequency for qubit 1, GHz
wd1 = wq1  # Drive frequency is the frequency for qubit 1
anharm0 = - 0.33 * (2 * numpy.pi)  # Anharmonicity of qubit 0, GHz
anharm1 = - 0.33 * (2 * numpy.pi)  # Anharmonicity of qubit 1, GHz

# Initialize the Hamiltonian
ham = QHam(subSysNum=qubits, sysLevel=level, dt=dt)

# Add the detuning terms
ham.addDrift(number, onSubSys=0, coef=wq0 - wd1)
ham.addDrift(number, onSubSys=1, coef=wq1 - wd1)

# Add the anharmonicity terms
ham.addDrift(duff, onSubSys=0, coef=anharm0 / 2)
ham.addDrift(duff, onSubSys=1, coef=anharm0 / 2)

# Add the coupling term
ham.addCoupling([0, 1], g=g / 2)

# Set amplitude bound
aBound = (1.0, 3.0)

# Run the optimization on Quanlse Cloud Service
job, infidelity = remoteOptimizeCr(ham, aBound=aBound, tg=200, maxIter=3, targetInfidelity=0.01)

## 动力学分析

研究量子操作过程中初始量子态的演化有助于我们理解控制脉冲对量子比特状态的影响。而量脉中的动力学分析功能可模拟给定初始状态下不同量子比特状态的演化。

在下面的示例中，我们将演示当初始量子比特状态为 $|01\rangle$ 时，执行 CR 门的过程中两比特计算空间内四个正交基（ $|00\rangle$，$|01\rangle$，$|10\rangle$ 和 $|11\rangle$）的演化过程。

首先，我们使用 `basis()` 函数为两比特系统定义完整的计算空间内的正交基，该函数的第一个输入是能级数（例如：三能级系统为 3），第二个输入是量子态（在这个示例中为 0 或 1 ）。然后，我们使用 `tensor()` 函数构造这两个量子比特的态矢量。

In [ ]:
# Define basis states: 00, 01, 10 and 11
state00 = tensor(basis(3, 0), basis(3, 0))
state01 = tensor(basis(3, 0), basis(3, 1))
state10 = tensor(basis(3, 1), basis(3, 0))
state11 = tensor(basis(3, 1), basis(3, 1))
stateList = [state00, state01, state10, state11]

随后，我们构造四个正交基所对应的投影矩阵，并根据这四个投影矩阵的期望值来分别计算每个正交基上的观测结果。我们使用 `projector()` 函数创建投影矩阵。`projector(a, b)` 的输入为两个先前定义的量子态向量（$|a\rangle$ 和 $|b\rangle$）并生成投影矩阵 $|a\rangle \langle b|$。如果仅将一个量子态向量作为输入，例如 $|a\rangle$，则此函数将返回 $|a\rangle \langle a|$。

In [ ]:
# Construct projection matrices from basis states
matrix00 = projector(state00)
matrix01 = projector(state01)
matrix10 = projector(state10)
matrix11 = projector(state11)
matrixList = [matrix00, matrix01, matrix10, matrix11]

接下来，我们使用 `Benchmark.evolution()` 函数来计算 CR 门操作过程中每个投影矩阵的期望值的变化过程。该函数需要三个输入参数：系统哈密顿量，初始态矢量列表和投影矩阵的列表。

In [ ]:
# Run the simulation to evaluate evolution of the expectation values
from Quanlse.Utils import Benchmark
ham.job = job
evolutionResult = Benchmark.evolution(ham, stateInitial=stateList, matrix=matrixList)

该函数的返回**结果**是一个 Python 字典。键 `0`，`1`，`2` … 指的是初始量子态的索引。该 Python 字典的每一个索引，都对应一个子 Python 字典，其键值包括：

* `state_form`：初始态矢量
* `state_evolution_history`：初始态矢量的演化过程详细数据
* `result`：不同投影矩阵期望值的演化数据。

现在，我们可以通过获取**不同基态**在**不同时刻**的期望值来绘制量子态演化的图像。在下面的部分中，我们将初始态定义为 $|01\rangle$，并计算概率与时间的演化关系。

In [ ]:
# Define x values to be the time for evolution
endTimeNs, endTimeDt = ham.job.computeMaxTime()
x = numpy.linspace(0, endTimeNs, endTimeDt)

# Define y values to be the expectation value for each projection matrix when the initial state is in 01, which corresponds to index 1
y1 = numpy.array(evolutionResult['1']['result']['matrix-0-value'])
y2 = numpy.array(evolutionResult['1']['result']['matrix-1-value'])
y3 = numpy.array(evolutionResult['1']['result']['matrix-2-value'])
y4 = numpy.array(evolutionResult['1']['result']['matrix-3-value'])

# Plot the population as absolute value of the expectation values
plt.plot(x, abs(y1), linewidth=3, label='00')
plt.plot(x, abs(y2), linewidth=3, label='01')
plt.plot(x, abs(y3), linewidth=3, label='10')
plt.plot(x, abs(y4), linewidth=3, label='11')
plt.title(r'Population Evolution for $|01\rangle$ Initial State ')
plt.xlabel('Time (ns)')
plt.ylabel('Population')
plt.legend()
plt.show()

## 真值表（Truth table）

量子门的真值表表示的是，不同的基态作为初始态在量子操作结束时处于不同基态的概率。在这里，我们从计算空间内选择初始态，而最终状态则可以处于计算空间之外，计算空间之外的末态即对应能级泄露。量脉提供了为量子过程计算真值表并使用热力图可视化结果的功能。它可以帮助我们分析能级泄露所带来的误差。

这里，我们将通过分析*构造哈密顿量并优化脉冲*章节生成的 CR 门的酉算符来介绍这个工具。

In [ ]:
# Import the function for generating the truth table
from Quanlse.Utils.Benchmark import stateTruthTable

# Import the function for plotting the heat map
from Quanlse.Utils.Plot import plotHeatMap

# Import the functions for basis operations
from Quanlse.Utils.Functions import generateBasisIndexList, computationalBasisList

# Indicate the input state list, and generate the list of the state indices
inputStateStr = ['00', '01', '10', '11']
initStateList = generateBasisIndexList(inputStateStr, level)

# Generate the matrix of the truth table
result = ham.simulate()
matrix = stateTruthTable(result[0]["unitary"], qubits, level, initStateList)

# Generate the list of the output state strings and plot the heat map
outputStateStr = computationalBasisList(qubits, level)
plotHeatMap(matrix, xTicks=outputStateStr, yTicks=inputStateStr, xLabel="Output State", yLabel="Input State", useLog=True)

通过上面的热力图，我们可以直观地读出演化以及能级泄露的信息。

## 总结

本教程介绍了量脉的误差分析功能。其中，动力学分析为我们提供了量子态演化的完整图像，而真值表则告诉我们不同初始状态下误差的分布。

在阅读了本教程之后，用户可以单击此链接 [tutorial-error-analysis.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-error-analysis-cn.ipynb) 到该 Jupyter Notebook 文档的 GitHub 页面，并运行该程序。我们鼓励用户在不同场景应用本教程提供的工具和方法。

## 参考文献

\[1\] [Nielsen, Michael A., and Isaac L. Chuang. Quantum Computation and Quantum Information: 10th Anniversary Edition. Cambridge University Press, 2010.](https://doi.org/10.1017/CBO9780511976667)